## This code is related to the Extereme Value Analysis

In [1]:
from scubas.datasets import PROFILES
from scubas.models import OceanModel
from scubas.plotlib import plot_transfer_function, potential_along_section, cable_potential, update_rc_params
from scubas.cables import TransmissionLine, Cable
from scubas.conductivity import ConductivityProfile as CP

land50 = PROFILES.CS_E
land50.layers[0].thickness = 50

def compile_cable_to_calculate_parameters(FRD_files, STJ_files, HAD_files):
    # Create cable by each cable sections
    tlines = []
    tlines.append(
        TransmissionLine(
            sec_id="CS-W",
            directed_length=dict(
                edge_locations=dict(
                    initial=dict(lat=39.6, lon=-74.33), 
                    final=dict(lat=38.79, lon=-72.62)
                )
            ),
            elec_params=dict(
                site=PROFILES.CS_W,
                width=1.0,
                flim=[1e-6, 1e0],
            ),
            active_termination=dict(
                right=None,
                left=PROFILES.LD,
            ),
        ).compile_oml(FRD_files),
    )
    tlines.append(
        TransmissionLine(
            sec_id="DO-1",
            directed_length=dict(
                edge_locations=dict(
                    initial=dict(lat=38.79, lon=-72.62), 
                    final=dict(lat=37.11, lon=-68.94)
                )
            ),
            elec_params=dict(
                site=PROFILES.DO_1,
                width=1.0,
                flim=[1e-6, 1e0],
            ),
            active_termination=dict(
                right=None,
                left=None,
            ),
        ).compile_oml(FRD_files)
    )
    tlines.append(
        TransmissionLine(
            sec_id="DO-2",
            directed_length=dict(
                edge_locations=dict(
                    initial=dict(lat=37.11, lon=-68.94), 
                    final=dict(lat=39.80, lon=-48.20)
                )
            ),
            elec_params=dict(
                site=PROFILES.DO_2,
                width=1.0,
                flim=[1e-6, 1e0],
            ),
            active_termination=dict(
                right=None,
                left=None,
            ),
        ).compile_oml(STJ_files)
    )
    tlines.append(
        TransmissionLine(
            sec_id="DO-3",
            directed_length=dict(
                edge_locations=dict(
                    initial=dict(lat=39.80, lon=-48.20), 
                    final=dict(lat=40.81, lon=-45.19)
                )
            ),
            elec_params=dict(
                site=PROFILES.DO_3,
                width=1.0,
                flim=[1e-6, 1e0],
            ),
            active_termination=dict(
                right=None,
                left=None,
            ),
        ).compile_oml(STJ_files)
    )
    tlines.append(
        TransmissionLine(
            sec_id="DO-4",
            directed_length=dict(
                edge_locations=dict(
                    initial=dict(lat=40.81, lon=-45.19), 
                    final=dict(lat=43.15, lon=-39.16)
                )
            ),
            elec_params=dict(
                site=PROFILES.DO_4,
                width=1.0,
                flim=[1e-6, 1e0],
            ),
            active_termination=dict(
                right=None,
                left=None,
            ),
        ).compile_oml(STJ_files)
    )
    tlines.append(
        TransmissionLine(
            sec_id="DO-5",
            directed_length=dict(
                edge_locations=dict(
                    initial=dict(lat=43.15, lon=-39.16), 
                    final=dict(lat=44.83, lon=-34.48)
                )
            ),
            elec_params=dict(
                site=PROFILES.DO_5,
                width=1.0,
                flim=[1e-6, 1e0],
            ),
            active_termination=dict(
                right=None,
                left=None,
            ),
        ).compile_oml(STJ_files)
    )
    tlines.append(
        TransmissionLine(
            sec_id="MAR",
            directed_length=dict(
                edge_locations=dict(
                    initial=dict(lat=44.83, lon=-34.48), 
                    final=dict(lat=46.51, lon=-22.43)
                )
            ),
            elec_params=dict(
                site=PROFILES.MAR,
                width=1.0,
                flim=[1e-6, 1e0],
            ),
            active_termination=dict(
                right=None,
                left=None,
            ),
        ).compile_oml(STJ_files)
    )
    tlines.append(
        TransmissionLine(
            sec_id="DO-6",
            directed_length=dict(
                edge_locations=dict(
                    initial=dict(lat=46.51, lon=-22.43), 
                    final=dict(lat=47.85, lon=-9.05)
                )
            ),
            elec_params=dict(
                site=PROFILES.DO_6,
                width=1.0,
                flim=[1e-6, 1e0],
            ),
            active_termination=dict(
                right=None,
                left=None,
            ),
        ).compile_oml(HAD_files)
    )
    tlines.append(
        TransmissionLine(
            sec_id="CS-E",
            directed_length=dict(
                edge_locations=dict(
                    initial=dict(lat=47.85, lon=-9.05), 
                    final=dict(lat=50.79, lon=-4.55)
                )
            ),
            elec_params=dict(
                site=PROFILES.CS_E,
                width=1.0,
                flim=[1e-6, 1e0],
            ),
            active_termination=dict(
                right=land50,
                left=None,
            ),
        ).compile_oml(HAD_files)
    )
    
    # Running the cable operation
    cable = Cable(tlines, tlines[0].components)
    return tlines, cable


Bad key "text.kerning_factor" on line 4 in
/home/shibaji/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
# Download the datasets, save to local, and validate datagaps

import numpy as np
import shutil
import requests
import os
import datetime as dt
import pandas as pd

clear = False
MAG_FOLDER, ELC_FOLDER = "ev/mag/", "ev/elc/"
if clear: 
    shutil.rmtree(MAG_FOLDER)
    shutil.rmtree(ELC_FOLDER)
os.makedirs(MAG_FOLDER, exist_ok=True)
os.makedirs(ELC_FOLDER, exist_ok=True)

BASE_URL = "https://imag-data.bgs.ac.uk/GIN_V1/hapi"

years, magnetometers, coord_keys = (
    range(2011,2021), ["stj", "frd", "had"], ["X", "Y", "Z"]
)

for mag in magnetometers:
    for year in years:
        dates = [
            dt.datetime(year, 1, 1),
            dt.datetime(year+1, 1, 1)
        ]
        expected_dp = (dates[1]-dates[0]).total_seconds() / 60.
        dates_str = [
            dates[0].strftime("%Y-%m-%dT%H:%M:%SZ"),
            dates[1].strftime("%Y-%m-%dT%H:%M:%SZ")
        ]
        link = f"/data?dataset={mag}/definitive/PT1M/native&parameters=Field_Vector&start={dates_str[0]}&stop={dates_str[1]}"
        url = BASE_URL + link
        fname = MAG_FOLDER + f"{mag.upper()}.{dates[0].year}.csv"
        # Download and save to file
        if not os.path.exists(fname):
            r = requests.get(url)
            print(f"Response code <{mag.upper()}>:", r.status_code)
            txt = "Date,X,Y,Z\n" + r.text
            with open(fname, "w") as f:
                f.write(txt)
            # Validate number of data points
            o = pd.read_csv(fname, parse_dates=["Date"])
            print(f"Expecetd / Actual # data points: {expected_dp} / {len(o)}")
            # Remove Datagaps
            o = o.replace(99999.0, np.nan)
            for key in coord_keys:                    
                o[key] = o[key].interpolate(method="from_derivatives")
                o[key] = o[key].ffill()
                o[key] = o[key].bfill()
            print("Is there any issue / nan data? ", o.isnull().values.any())
            # Baseline removal
            for key in coord_keys:
                fmed = np.nanmedian(o[key][:120])
                o[key] = o[key] - fmed
            o.to_csv(fname, header=True, index=False, float_format="%g")

for year in years:            
    FRD_files, STJ_files, HAD_files = (
        [MAG_FOLDER + f"FRD.{year}.csv"],
        [MAG_FOLDER + f"STJ.{year}.csv"],
        [MAG_FOLDER + f"HAD.{year}.csv"]
    )
    fname = ELC_FOLDER + f"{year}.csv"
    if not os.path.exists(fname):
        tlines, cable = compile_cable_to_calculate_parameters(FRD_files, STJ_files, HAD_files)
        o = cable.tot_params.copy().reset_index()
        o.to_csv(fname, header=True, float_format="%g")
    else:
        o = pd.read_csv(fname, parse_dates=["Time"])
    o["hour"] = o.Time.apply(lambda x: x.hour + (x.dayofyear-1)*24)
    parameters = [
        "hour", "Time", "Vt(v)", "V(v)",
        "V(v).00", "V(v).01", "V(v).02",
        "V(v).03", "V(v).04", "V(v).05",
        "V(v).06", "V(v).07", "V(v).08",
        "U0", "U1", "E.Y", "E.Y.00", 
        "E.Y.01", "E.Y.02", "E.Y.03",
        "E.Y.04", "E.Y.05", "E.Y.06",
        "E.Y.07", "E.Y.08", "E.X", "E.X.00",
        "E.X.01", "E.X.02", "E.X.03",
        "E.X.04", "E.X.05", "E.X.06", "E.X.07", "E.X.08"
    ]
    o = o[parameters]
    for p in parameters[2:]:
        o[p] = o[p].abs()
    o["V_W"] = o["V(v).00"] + o["V(v).01"] + o["U0"]
    o["V_E"] = o["V(v).07"] + o["V(v).08"] - o["U1"]
    o["V_M"] = o["V(v).02"] + o["V(v).03"] + o["V(v).04"] + o["V(v).05"] + o["V(v).06"]
    parameters.extend(["V_W", "V_E", "V_M"])
    o = o.groupby(by="hour").agg([np.nanmin, np.nanmax])
    df = pd.DataFrame()
    df["start_time"], df["end_time"] = o.Time["nanmin"], o.Time["nanmax"]
    for p in parameters[2:]:
        (df[f"{p.replace('(v)', '')}_min"], df[f"{p.replace('(v)', '')}_max"]) = (
            o[p]["nanmin"], o[p]["nanmax"]
        )
    df.to_csv(fname.replace(".csv", ".hr.csv"), header=True, index=False, float_format="%g")

In [3]:
FRD_files, STJ_files, HAD_files = (
        [
            "input/data/1989/FRD_19890312_XYZ.txt", 
            "input/data/1989/FRD_19890313_XYZ.txt", 
            "input/data/1989/FRD_19890314_XYZ.txt"
        ],
        [
            "input/data/1989/STJ_19890312_XYZ.txt", 
            "input/data/1989/STJ_19890313_XYZ.txt", 
            "input/data/1989/STJ_19890314_XYZ.txt"
        ],
        [
            "input/data/1989/HAD_19890312_HDZ.txt", 
            "input/data/1989/FRD_19890313_HDZ.txt", 
            "input/data/1989/FRD_19890314_HDZ.txt"
        ]
    )
tlines, cable = compile_cable_to_calculate_parameters(FRD_files, STJ_files, HAD_files)

2023-10-30 14:16:19.306 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width CS-W: 1.0
2023-10-30 14:16:19.306 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Continental Shelf West E- and B-Fields
2023-10-30 14:16:19.624 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width CS-W: 1.0
2023-10-30 14:16:19.637 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width DO-1: 1.0
2023-10-30 14:16:19.637 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Deep Ocean E- and B-Fields
2023-10-30 14:16:19.720 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width DO-2: 1.0
2023-10-30 14:16:19.720 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Deep Ocean E- and B-Fields
2023-10-30 14:16:19.828 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width DO-3: 1.0
2023-10-30 14:16:19.828 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Dee

In [24]:
# Download the datasets, save to local, and validate datagaps

import numpy as np
import shutil
import requests
import os
import datetime as dt
import pandas as pd

ELC_FOLDER = "ev/elc/t/"
os.makedirs(ELC_FOLDER, exist_ok=True)

years, magnetometers, coord_keys = (
    range(1991,2021), ["stj", "frd", "had"], ["X", "Y", "Z"]
)
for sec_id, site in zip(
    ["DO-2", "DO-3", "DO-4", "DO-5", "MAR", "DO-6", "CS_E"],
    [PROFILES.DO_2, PROFILES.DO_3, PROFILES.DO_4, PROFILES.DO_5, PROFILES.MAR, PROFILES.DO_6, PROFILES.CS_E]
):
    stn = "STJ"
    for year in years:    
        mag_file_name = [MAG_FOLDER + f"{stn}.{year}.csv"]
        fname = ELC_FOLDER + f"{year}-{sec_id.replace('-', '_')}-{stn}-hr.csv"
        if not os.path.exists(fname):
            tl = TransmissionLine(
                sec_id=sec_id,
                directed_length=dict(
                edge_locations=dict(
                    initial=dict(lat=39.6, lon=-74.33), 
                    final=dict(lat=38.79, lon=-72.62)
                    )
                ),
                elec_params=dict(
                    site=site,
                    width=1.0,
                    flim=[1e-6, 1e0],
                ),
                active_termination=dict(
                    right=None,
                    left=None,
                ),
            ).compile_oml(mag_file_name)
            o = tl.Efield.copy().reset_index()
            o["A"] = np.sqrt(o.X**2+o.Y**2)
            o["hour"] = o.Time.apply(lambda x: x.hour + (x.dayofyear-1)*24)
            o = o.groupby(by="hour").agg([np.nanmin, np.nanmax])
            df = pd.DataFrame()
            df["start_time"], df["end_time"] = o.Time["nanmin"], o.Time["nanmax"]
            for p in ["X","Y","A"]:
                (df[f"{p}_min"], df[f"{p}_max"]) = (
                    o[p]["nanmin"], o[p]["nanmax"]
                )
            df.to_csv(fname, header=True, index=False, float_format="%g")


2023-10-30 15:46:32.159 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width DO-2: 1.0
2023-10-30 15:46:32.160 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Deep Ocean E- and B-Fields
2023-10-30 15:46:38.381 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width DO-2: 1.0
2023-10-30 15:46:38.381 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Deep Ocean E- and B-Fields
2023-10-30 15:46:44.491 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width DO-2: 1.0
2023-10-30 15:46:44.491 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Deep Ocean E- and B-Fields
2023-10-30 15:46:50.606 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width DO-2: 1.0
2023-10-30 15:46:50.606 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Deep Ocean E- and B-Fields
2023-10-30 15:46:56.944 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable widt

2023-10-30 15:50:14.075 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Deep Ocean E- and B-Fields
2023-10-30 15:50:20.145 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width DO-3: 1.0
2023-10-30 15:50:20.145 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Deep Ocean E- and B-Fields
2023-10-30 15:50:26.199 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width DO-3: 1.0
2023-10-30 15:50:26.199 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Deep Ocean E- and B-Fields
2023-10-30 15:50:32.332 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width DO-3: 1.0
2023-10-30 15:50:32.333 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Deep Ocean E- and B-Fields
2023-10-30 15:50:38.485 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width DO-3: 1.0
2023-10-30 15:50:38.486 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Deep Ocean E- 

2023-10-30 15:54:02.874 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width DO-4: 1.0
2023-10-30 15:54:02.875 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Deep Ocean E- and B-Fields
2023-10-30 15:54:09.056 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width DO-4: 1.0
2023-10-30 15:54:09.057 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Deep Ocean E- and B-Fields
2023-10-30 15:54:15.136 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width DO-4: 1.0
2023-10-30 15:54:15.136 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Deep Ocean E- and B-Fields
2023-10-30 15:54:21.224 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width DO-4: 1.0
2023-10-30 15:54:21.225 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Deep Ocean E- and B-Fields
2023-10-30 15:54:27.258 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable widt

2023-10-30 15:57:44.250 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Deep Ocean E- and B-Fields
2023-10-30 15:57:50.313 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width DO-5: 1.0
2023-10-30 15:57:50.314 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Deep Ocean E- and B-Fields
2023-10-30 15:57:56.293 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width DO-5: 1.0
2023-10-30 15:57:56.294 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Deep Ocean E- and B-Fields
2023-10-30 15:58:02.491 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width DO-5: 1.0
2023-10-30 15:58:02.491 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Deep Ocean E- and B-Fields
2023-10-30 15:58:08.566 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width DO-5: 1.0
2023-10-30 15:58:08.566 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Deep Ocean E- 

2023-10-30 16:01:24.691 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Mid-Atlantic Ridge E- and B-Fields
2023-10-30 16:01:30.862 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width MAR: 1.0
2023-10-30 16:01:30.862 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Mid-Atlantic Ridge E- and B-Fields
2023-10-30 16:01:36.952 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width MAR: 1.0
2023-10-30 16:01:36.952 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Mid-Atlantic Ridge E- and B-Fields
2023-10-30 16:01:43.084 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width MAR: 1.0
2023-10-30 16:01:43.084 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Mid-Atlantic Ridge E- and B-Fields
2023-10-30 16:01:49.111 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width MAR: 1.0
2023-10-30 16:01:49.111 | INFO     | scubas.models:__init__:44 - Compile

2023-10-30 16:05:11.479 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width CS_E: 1.0
2023-10-30 16:05:11.479 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Continental Shelf East E- and B-Fields
2023-10-30 16:05:17.495 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width CS_E: 1.0
2023-10-30 16:05:17.496 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Continental Shelf East E- and B-Fields
2023-10-30 16:05:23.507 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width CS_E: 1.0
2023-10-30 16:05:23.508 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Continental Shelf East E- and B-Fields
2023-10-30 16:05:29.574 | INFO     | scubas.cables:calc_trasmission_line_parameters:214 - Cable width CS_E: 1.0
2023-10-30 16:05:29.574 | INFO     | scubas.models:__init__:44 - Compile Ocean-model: Continental Shelf East E- and B-Fields
2023-10-30 16:05:35.612 | INFO     | scubas.cables:c